In [ ]:
"""
Bicicletas recorridos
Clima PRCP TAVG
2020-2021
acquisition clean prepare upload
"""
import os
import sys

import urllib.request
# from zipfile import ZipFile

import pandas as pd
import numpy as np

import random

# Mover esto a exploring
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib as mpl

# mpl.rcParams.update( mpl.rcParamsDefault ) 

In [ ]:
bicicletas = {
    "recorridos" : {
        "2020" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2020.zip',
            "zip_file" : "./datasets/recorridos-realizados-2020.zip",
            "df" : pd.DataFrame(),
        },
        "2021" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2021.zip',   
            "zip_file" : "./datasets/recorridos-realizados-2021.zip",
            "df" : pd.DataFrame(),
        },
    },
    "recorridos_df" : pd.DataFrame(),
    "estaciones" : pd.DataFrame(),
}

In [ ]:
"""
Descargar recorridios bicicletas
"""
def descargar_recoridos(
    recorridos,
    equivalencias_encabezados
):
    for r in recorridos:
        if not os.path.exists( recorridos[r]["zip_file"] ):
            print("Descargando", recorridos[r]["url"])
            urllib.request.urlretrieve(
              recorridos[r]["url"],
              recorridos[r]["zip_file"]
            )
        else:
            print("Ya descargado", recorridos[r]["zip_file"])  
        recorridos[r]["df"] = pd.read_csv(
            recorridos[r]["zip_file"],
            compression = 'zip',
            encoding = 'utf-8', 
        )
#         print( recorridos[r]["df"].info() )
    # ajustar 2021
    recorridos["2021"]["df"] = recorridos["2021"]["df"]
    recorridos["2021"]["df"] = recorridos["2021"]["df"].rename( columns = equivalencias_encabezados )
    recorridos["2021"]["df"]["id"] = recorridos["2021"]["df"]['id'].astype('Int64')
    print('done')
    return recorridos

equivalencias_encabezados = {
        'ID'                                  : 'id',
        'Duración'                            : 'duracion_recorrido',
        'Estado cerrado'                      : 'estado_cerrado',
        'ID de ciclista'                      : 'id_usuario',
        'Tipo de ciclista'                    : 'cilicsta_tipo',
        'Modelo de bicicleta'                 : 'bicicleta_modelo',   

        'Id de estación de inicio'            : 'id_estacion_origen', 
        'Nombre de estación de inicio'        : 'nombre_estacion_origen',
        'Fecha de inicio'                     : 'fecha_origen_recorrido',     
        'Origen de viaje'                     : 'direccion_estacion_origen',  
        # '':'long_estacion_origen',        
        # '':'lat_estacion_origen',
        'Id de estación de fin de viaje'      : 'id_estacion_destino',     
        'Nombre de estación de fin de viaje'  : 'nombre_estacion_destino',
        'Fecha de fin'                        : 'fecha_destino_recorrido',          
        # '':'direccion_estacion_destino',           
        # '':'long_estacion_destino',       
        # '':'lat_estacion_destino',
        # '':'periodo',  
}
bicicletas["recorridos"] = descargar_recoridos(
    bicicletas["recorridos"],
    equivalencias_encabezados
)

In [ ]:
bicicletas["recorridos"]["2021"]["df"].info()
bicicletas["recorridos"]["2020"]["df"].info()

In [ ]:
# Concatenar recorridos 2020 2021
frames = []
for r in bicicletas["recorridos"]:
    frames.append( bicicletas["recorridos"][r]["df"] )
bicicletas["recorridos_df"] = pd.concat( frames )

In [ ]:
bicicletas["recorridos_df"] 

In [ ]:
# Fill nan en id y destinos
# bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].interpolate( method='linear' ) # no funco
bicicletas["recorridos_df"] = bicicletas["recorridos_df"].sort_values( by=['fecha_origen_recorrido'] )
bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].fillna(0)
bicicletas["recorridos_df"]['nombre_estacion_destino'] = bicicletas["recorridos_df"]['nombre_estacion_destino'].fillna('0 - null')

In [ ]:
# Filtro y ordeno columnas que quedan
orden_filtro = [
    'id',
    'duracion_recorrido',
#     'estado_cerrado',
    'id_usuario',
#     'cilicsta_tipo',
#     'bicicleta_modelo',
    'id_estacion_origen',
    'nombre_estacion_origen',       
#     'direccion_estacion_origen',
    'fecha_origen_recorrido',       
#     'long_estacion_origen',     
#     'lat_estacion_origen',
    'id_estacion_destino',         
    'nombre_estacion_destino',    
#     'direccion_estacion_destino',
    'fecha_destino_recorrido',      
#     'long_estacion_destino',       
#     'lat_estacion_destino', 
#     'periodo',                       
]
bicicletas["recorridos_df"] = bicicletas["recorridos_df"][orden_filtro]

In [ ]:
bicicletas["recorridos_df"]

### Procesar fecha

Parsear object datetime a dia, mes, año

In [ ]:
# bicis_recorridos_2020_df['fecha_origen_recorrido']
formato = '%Y%m%d %H:%M:%S'
bicicletas["recorridos_df"]['start_datetime'] = pd.to_datetime(
    bicicletas["recorridos_df"]['fecha_origen_recorrido'], 
    format = formato
)
# bicicletas["recorridos_df"]['start_date'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y-%m-%d')
bicicletas["recorridos_df"]['anio'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y')
bicicletas["recorridos_df"]['nmes'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%m')
# bicicletas["recorridos_df"]['mes']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%b').str.lower()
bicicletas["recorridos_df"]['ndia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%j')
# bicicletas["recorridos_df"]['dia']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%a').str.lower()
bicicletas["recorridos_df"]['wdia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%w')
bicicletas["recorridos_df"]['hora'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%H')
bicicletas["recorridos_df"]['mins'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%M')
bicicletas["recorridos_df"]

In [ ]:
# Separar las columnas nombre de estacion de origen y nombre de estacion de destino 
# Tienen la info correcta del id de las estaciones

# Use vectoried str.split with expand = True:
bicicletas["recorridos_df"][
    ['id_estacion_incio','nombre_estacion_origen']
] = bicicletas["recorridos_df"]['nombre_estacion_origen'].str.split( ' - ', expand = True )

# Aca divido la columna Nombre de estacion de fin de viaje porque contiene dos campos 
# el Id y el nombre de la estacion de inicio de viaje.
# registros Nan ya pàsados a '0 - null'
bicicletas["recorridos_df"][
    ['id_estacion_destino','nombre_estacion_destino']
] = bicicletas["recorridos_df"]['nombre_estacion_destino'].str.split( ' - ', expand = True )
# TODO: sacar los leading zeros

# bicicletas["recorridos_df"]["id_estacion_incio"] = bicicletas["recorridos_df"]["id_estacion_incio"].astype('Int64')
bicicletas["recorridos_df"]

In [ ]:
bicicletas["recorridos_clima_df"] = pd.merge(
    left = bicicletas["recorridos_df"],
    right = weather_data_df,
    left_on = 'start_datetime',
    right_on = 'DATE'
)

In [ ]:
# Exportar CSV

In [14]:
compression_opts = dict(
    method='zip',
    archive_name='./datasets/recorridos_2020-2021.csv'
)  

bicicletas["recorridos_df"].to_csv(
    './datasets/recorridos_2020-2021.zip',
    index = False,
    compression = compression_opts
)

# Load to GoogleDrive

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# Fetch weather from drive
id = '14BpBfjQ3IrfeK8CajCWrEdfPg7S0eqc8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('weather-data-ba.csv') 
weather_data_df = pd.read_csv('weather-data-ba.csv')
weather_data_df.head()
weather_data_df = weather_data_df.filter(
    ['DATE','PRCP','TAVG']
)
weather_data_df.isnull().sum( axis = 0 )
weather_data_df['PRCP'] = weather_data_df['PRCP'].replace( np.nan, 0 )
weather_data_df.head()

In [ ]:
bicicletas["recorridos_clima_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos-clima-2020-2021.csv'
)

In [ ]:
bicicletas["recorridos_clima_df"].head()

## Bajar el ultimo
Bajar el dataset limpio desde drive

In [ ]:
from google.colab import files
files.download('recorridos_2020-2021.zip')

In [ ]:
bicicletas["recorridos_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos_2020-2021.csv'
)

In [ ]:
id = '17liAPD1lyj2wT-Hcbh4ed9-Cn4peoyIq'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('recorridos_2020-2021.csv') 
bicicletas["recorridos_df"] = pd.read_csv('recorridos_2020-2021.csv')
bicicletas["recorridos_df"].head()